In [ ]:
import argparse
import os
import warnings
import yaml

import pytorch_lightning as pl
import torch
from pytorch_lightning.callbacks import TQDMProgressBar
from pytorch_lightning.accelerators import find_usable_cuda_devices
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader
from nanodet.data.collate import naive_collate
from nanodet.data.dataset import build_dataset
from nanodet.evaluator import build_evaluator
from nanodet.trainer.task import TrainingTask
from torchvision.transforms import ToTensor, ToPILImage
from nanodet.util import (
    NanoDetLightningLogger,
    cfg,
    convert_old_model,
    env_utils,
    load_config,
    load_model_weight,
    mkdir,
)

#Set logger and seed
logger = NanoDetLightningLogger('test')
pl.seed_everything(1234)

In [ ]:
#Function to create the task configuration file required for training
def create_exp_cfg(yml_path, task):
    all_names = ["aeroplane", "bicycle", "bird", "boat", "bottle", "bus", "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike", "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]
    #Load the YAML file
    with open(yml_path, 'r') as file:
        temp_cfg = yaml.safe_load(file)
    #Save dir of the model
    temp_cfg['save_dir'] = 'models/task' + str(task)
    #If base task, training and testing classes are the same
    if task == 0:
        temp_cfg['data']['train']['class_names'] = all_names[:15]
        temp_cfg['data']['val']['class_names'] = all_names[:15]
        temp_cfg['model']['arch']['head']['num_classes'] = 20 #15
        #temp_cfg['model']['arch']['aux_head']['num_classes'] = 20 #15
    #Else, training only on task specific class, and testing on all classes
    else:
        temp_cfg['data']['train']['class_names'] = [all_names[14+task]]
        temp_cfg['data']['val']['class_names'] = all_names[:15+task]
        temp_cfg['model']['arch']['head']['num_classes'] = 20#15+task
        #temp_cfg['model']['arch']['aux_head']['num_classes'] = 20#15+task
        temp_cfg['schedule']['load_model'] = 'models/task' + str(task-1) + '/model_last.ckpt'
        
    temp_cfg_name = 'cfg/task' + str(task) + '.yml'
    print(temp_cfg_name)
    #Save the new configuration file
    with open(temp_cfg_name, 'w') as file:
        yaml.safe_dump(temp_cfg, file)

In [ ]:
#Learning stream
#task 0: train on first 15 classes, test on 15 classes
#task 1: train on class n°16, test on 16 classes
#task 2: train on class n°17, test on 17 classes
#task 3: train on class n°18, test on 18 classes
#task 4: train on class n°19, test on 19 classes
#task 5: train on class n°20, test on 20 classes
for task in range (1, 6):
    logger = NanoDetLightningLogger('run_logs/task'+str(task))
    logger.info("Starting task" + str(task))
    logger.info("Setting up data...")
    #Create the task configuration file based on the task number and load the configuration
    create_exp_cfg('cfg/VOC.yml', task)
    load_config(cfg, 'cfg/task' + str(task) + '.yml')
    #Build datasets and dataloaders based on the task configuration file
    train_dataset = build_dataset(cfg.data.train, "train")
    val_dataset = build_dataset(cfg.data.val, "test")
    evaluator = build_evaluator(cfg.evaluator, val_dataset)
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=cfg.device.batchsize_per_gpu,
        shuffle=True,
        num_workers=cfg.device.workers_per_gpu,
        pin_memory=True,
        collate_fn=naive_collate,
        drop_last=True,
    )
    val_dataloader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=cfg.device.batchsize_per_gpu,
        shuffle=False,
        num_workers=cfg.device.workers_per_gpu,
        pin_memory=True,
        collate_fn=naive_collate,
        drop_last=False,
    )
    #Create the model based on the task configuration file
    logger.info("Creating model...")
    task = TrainingTask(cfg, evaluator)
    #Load the model weights if task is not 0
    if "load_model" in cfg.schedule:
        ckpt = torch.load(cfg.schedule.load_model)
        if "pytorch-lightning_version" not in ckpt:
            warnings.warn(
                "Warning! Old .pth checkpoint is deprecated. "
                "Convert the checkpoint with tools/convert_old_checkpoint.py "
            )
            ckpt = convert_old_model(ckpt)
        load_model_weight(task.model, ckpt, logger)
        logger.info("Loaded model weight from {}".format(cfg.schedule.load_model))
    model_resume_path = (
        os.path.join(cfg.save_dir, "model_last.ckpt")
        if "resume" in cfg.schedule
        else None
    )
    #Set the device to GPU if available
    if cfg.device.gpu_ids == -1:
        logger.info("Using CPU training")
        accelerator, devices, strategy, precision = (
            "cpu",
            None,
            None,
            cfg.device.precision,
        )
    else:
        accelerator, devices, strategy, precision = (
            "gpu",
            cfg.device.gpu_ids,
            None,
            cfg.device.precision,
        )

    if devices and len(devices) > 1:
        strategy = "ddp"
        env_utils.set_multi_processing(distributed=True)

    trainer = pl.Trainer(
        default_root_dir=cfg.save_dir,
        max_epochs=cfg.schedule.total_epochs,
        check_val_every_n_epoch=cfg.schedule.val_intervals,
        accelerator=accelerator,
        devices=[2],
        log_every_n_steps=cfg.log.interval,
        num_sanity_val_steps=0,
        callbacks=[TQDMProgressBar(refresh_rate=0)],
        logger=logger,
        benchmark=cfg.get("cudnn_benchmark", True),
        gradient_clip_val=cfg.get("grad_clip", 0.0),
        strategy=strategy,
        precision=precision,
    )
    trainer.fit(task, train_dataloader, val_dataloader, ckpt_path=model_resume_path)

In [ ]:
#Learning stream
#task 0: train on first 15 classes, test on 15 classes
#task 1: train on class n°16, test on 16 classes
#task 2: train on class n°17, test on 17 classes
#task 3: train on class n°18, test on 18 classes
#task 4: train on class n°19, test on 19 classes
#task 5: train on class n°20, test on 20 classes


#Create the task configuration file based on the task number and load the configuration
load_config(cfg, 'cfg/VOCsingle.yml')
#Build datasets and dataloaders based on the task configuration file
train_dataset = build_dataset(cfg.data.train, "train")
val_dataset = build_dataset(cfg.data.val, "test")
evaluator = build_evaluator(cfg.evaluator, val_dataset)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=cfg.device.batchsize_per_gpu,
    shuffle=True,
    num_workers=cfg.device.workers_per_gpu,
    pin_memory=True,
    collate_fn=naive_collate,
    drop_last=True,
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=cfg.device.batchsize_per_gpu,
    shuffle=False,
    num_workers=cfg.device.workers_per_gpu,
    pin_memory=True,
    collate_fn=naive_collate,
    drop_last=False,
)
#Create the model based on the task configuration file
logger.info("Creating model...")
task = TrainingTask(cfg, evaluator)
#Load the model weights if task is not 0
if "load_model" in cfg.schedule:
    ckpt = torch.load(cfg.schedule.load_model)
    if "pytorch-lightning_version" not in ckpt:
        warnings.warn(
            "Warning! Old .pth checkpoint is deprecated. "
            "Convert the checkpoint with tools/convert_old_checkpoint.py "
        )
        ckpt = convert_old_model(ckpt)
    load_model_weight(task.model, ckpt, logger)
    logger.info("Loaded model weight from {}".format(cfg.schedule.load_model))
model_resume_path = (
    os.path.join(cfg.save_dir, "model_last.ckpt")
    if "resume" in cfg.schedule
    else None
)
#Set the device to GPU if available
if cfg.device.gpu_ids == -1:
    logger.info("Using CPU training")
    accelerator, devices, strategy, precision = (
        "cpu",
        None,
        None,
        cfg.device.precision,
    )
else:
    accelerator, devices, strategy, precision = (
        "gpu",
        cfg.device.gpu_ids,
        None,
        cfg.device.precision,
    )

if devices and len(devices) > 1:
    strategy = "ddp"
    env_utils.set_multi_processing(distributed=True)

trainer = pl.Trainer(
    default_root_dir=cfg.save_dir,
    max_epochs=cfg.schedule.total_epochs,
    check_val_every_n_epoch=10,
    accelerator=accelerator,
    devices=[2],
    log_every_n_steps=cfg.log.interval,
    num_sanity_val_steps=0,
    callbacks=[TQDMProgressBar(refresh_rate=0)],
    logger=logger,
    benchmark=cfg.get("cudnn_benchmark", True),
    gradient_clip_val=cfg.get("grad_clip", 0.0),
    strategy=strategy,
    precision=precision,
)
trainer.fit(task, train_dataloader, val_dataloader, ckpt_path=model_resume_path)

In [ ]:
###############
# PAPER PLOTS #
###############

# Assuming your method is the last one in the list
from matplotlib import pyplot as plt
import numpy as np
from matplotlib import rc
import matplotlib

strategy = ['Fine-Tuning', 'Classic Replay', 'LwF', 'SID', 'Latent Distillation (ours)']
plt.rcParams.update({'font.size': 14})
# Use different markers for different methods
markers = ['o', 'o', 'o', 'o', 'o']
colors = ['purple', 'blue', 'orange', 'green', 'red']

plt.figure(figsize=(10, 5))
memory = [1.2, 1.2, 2.4, 2.4, 1.59]
mAP = [3.2, 16.3, 21.7, 22.8, 23.6]

# Parameter that determines the size of the dots
size_parameter = [200, 200, 200, 200, 200]

for i in range(len(strategy)):
    plt.scatter(memory[i], mAP[i], marker=markers[i], color=colors[i], label=strategy[i], s=size_parameter[i], zorder=2)

plt.grid(True, zorder=1)

plt.xlabel('Total Parameters (Million)')
plt.ylabel('Overall mAP@[0.50:0.95]')
plt.xticks()
plt.title('Total Parameters vs Overall mAP@[0.50:0.95] on scenario 19p1')
plt.legend(title='CL strategy', loc='lower right')
plt.savefig('mAPvsMem.eps', format='eps')
plt.show()

training_sets = ['base 15', '+ pottedplant', '+ sheep', '+ sofa', '+ train', '+ tvmonitor']
strategy = ['Naive', 'Classic Replay', 'LwF', 'SID', 'Latent Distillation (ours)']
colors = ['purple', 'blue', 'orange', 'green', 'red']
from matplotlib import pyplot as plt
import numpy as np
from matplotlib import rc
import matplotlib

# Use LaTeX for text and set font size
plt.rcParams.update({'font.size': 14})
auc_values = [[0.319, 0.07276294154642317, 0.017718051470568385, 0.007125799587737094, 0.00463388840764482, 0.0064597139891013665],
              [0.319, 0.203, 0.176, 0.101, 0.077, 0.050],
              [0.319, 0.206, 0.121, 0.028, 0.007, 0.003],
              [0.319, 0.204, 0.144, 0.156, 0.104, 0.101],
              [0.319, 0.228, 0.173, 0.148, 0.11, 0.096]]
# Plot
plt.figure(figsize=(10, 5))
for i in range(0,5):
    plt.plot(training_sets, auc_values[i], marker='o',color=colors[i], label=strategy[i])
plt.xlabel('Task')
plt.ylabel('mAP@[0.50:0.95]')
plt.xticks(np.arange(len(training_sets)), training_sets)
plt.title('Task stream on VOC2007 scenario 15p1')
plt.legend(title='CL strategy', loc='upper right') #box_to_anchor=(1.05, 1),
plt.grid(True)
plt.savefig('mAP_15p1.eps', format='eps')
plt.show()